# Mech Tools Evaluation

### Create the dataset

In [1]:
# !docker build -t etl:0.0.1 . && docker run etl:0.0.1

### Load the dataset

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.precision", 2)
pd.set_option('display.max_columns', None)

In [3]:
dataset = pd.read_csv("./data/dataset.csv")
# drop duplicates
dataset = dataset.drop_duplicates(subset='request_id')

# choose only non error
dataset = dataset[dataset['error'] == False].reset_index(drop=True)

str_cols = ("id", "currentAnswer", "title", "request_id", "prompt_request", "tool", "nonce", "vote")
for col in str_cols:
    dataset[col] = dataset[col].astype("string")
dataset.head()



,id,currentAnswer,title,request_id,request_block,prompt_request,tool,nonce,deliver_block,error,error_message,prompt_response,p_yes,p_no,confidence,info_utility,vote,win_probability
0,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,2315482717523248142566209479998403158981026548...,29555552,"With the given question ""Will the Hisense U8K ...",prediction-online,2a54469c-a055-4c9c-a558-51262cd4a787,29557480,False,NaN,NaN,0.50,0.50,0.8,0.6,<NA>,0.50
1,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,3469031602277041443388090267865736162834108248...,29555700,"With the given question ""Will the Hisense U8K ...",prediction-online,01538074-5f78-4e28-8058-47b6d94b682f,29557255,False,NaN,NaN,0.60,0.40,0.8,0.6,Yes,0.60
2,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,9159320707509526863870444110528895654334237837...,29555970,"Given the information available, and consideri...",prediction-online,099e342e-a41e-4689-be76-3570d8a256cf,29556974,False,NaN,NaN,0.65,0.35,0.8,0.6,Yes,0.65
3,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,1023057355467695172592840970123356496552194251...,29556083,"With the given question ""Will the Hisense U8K ...",prediction-online,438e9446-0f55-4e5e-b879-c06e93520969,29556905,False,NaN,NaN,0.60,0.40,0.8,0.7,Yes,0.60
4,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,1049080869663393036021703581760667415975346534...,29556451,"With the given question ""Will the Hisense U8K ...",prediction-online,7209cbcf-030e-4664-956f-a1882c3d35c2,29556491,False,NaN,NaN,0.60,0.40,0.8,0.5,Yes,0.60


In [4]:
dataset.shape

(67917, 18)

In [5]:
dataset.describe()

,request_block,deliver_block,p_yes,p_no,confidence,info_utility,win_probability
count,6.79e+04,6.79e+04,67917.00,67917.00,67917.00,67917.00,67917.00
mean,3.07e+07,3.07e+07,0.48,0.52,0.78,0.56,0.67
std,6.75e+05,6.74e+05,0.20,0.20,0.08,0.18,0.10
min,2.89e+07,2.89e+07,0.00,0.00,0.00,0.00,0.50
25%,3.01e+07,3.01e+07,0.30,0.40,0.80,0.50,0.60
50%,3.06e+07,3.06e+07,0.60,0.40,0.80,0.60,0.70
75%,3.12e+07,3.12e+07,0.60,0.70,0.80,0.60,0.75
max,3.20e+07,3.20e+07,1.00,1.00,1.00,1.00,1.00


In [6]:
dataset["vote"].describe()

count     62930
unique        2
top         Yes
freq      34688
Name: vote, dtype: object

### Normalize confidences

In [7]:
dataset["confidence"].unique()

array([0.8 , 0.9 , 0.7 , 0.6 , 0.5 , 0.1 , 0.75, 0.85, 0.3 , 1.  , 0.35,
       0.2 , 0.4 , 0.65, 0.99, 0.95, 0.55, 0.  , 0.62, 0.88, 0.64, 0.81,
       0.68])

In [8]:
# number of samples with confidence below 0.5
print(f"Number of samples with confidence below 0.5: {dataset[dataset['confidence'] < 0.5].shape[0]}")

# drop samples with confidence below 0.5
dataset = dataset[dataset['confidence'] >= 0.5].reset_index(drop=True)

Number of samples with confidence below 0.5: 38


In [9]:
# bucket confidence into 0.5, 0.6, 0.7, 0.8, 0.9
dataset['confidence'] = dataset['confidence'].apply(lambda x: round(x, 1))

# if confidence is 1 --> 0.9
dataset['confidence'] = dataset['confidence'].apply(lambda x: 0.9 if x == 1.0 else x)
dataset["confidence"].unique()

array([0.8, 0.9, 0.7, 0.6, 0.5])

### Check the percentage of wins vs confidence for all the tools

In [10]:
def accuracy(data):
    correct_answers_mask = data["currentAnswer"] == data["vote"]
    n_answers = correct_answers_mask.count()
    n_answers_success = correct_answers_mask.sum()

    if n_answers == 0:
        accuracy = None
    elif n_answers_success == 0:
        accuracy = 0
    else:
        accuracy = n_answers_success/n_answers * 100

    return pd.Series({"n_correct": n_answers_success, "n_pred": n_answers, "accuracy": accuracy})

In [11]:
def acc_per_tool(group, col: str, conf: float):
    return group.apply(lambda x: accuracy(x[x[col] == conf]))

In [12]:
def gen_stats(group, col: str):
    stats = {f"{col}_{prob}" : acc_per_tool(group, col, prob) for prob in sorted(dataset[col].unique())}
    stats["total"] = tools_group.apply(accuracy)
    return pd.concat(stats.values(), axis=1, keys=stats.keys())

In [13]:
tools_group = dataset.groupby("tool")
tools_stats_per_conf = gen_stats(tools_group, "confidence")
display(tools_stats_per_conf)

confidence_0.5                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                           597.0  1017.0    58.70   
claude-prediction-online                            186.0   384.0    48.44   
prediction-offline                                    0.0     0.0      NaN   
prediction-offline-sme                                0.0     0.0      NaN   
prediction-online                                     0.0     0.0      NaN   
prediction-online-sme                                 1.0     3.0    33.33   
prediction-online-summarized-info                     0.0     0.0      NaN   
prediction-sentence-embedding-bold                    0.0     0.0      NaN   
prediction-sentence-embedding-conservative            0.0     0.0      NaN   

                                           confidence_0.6                  \
                                                n_correct n_pred accuracy   
tool                                                                        
claude-prediction-offline                            50.0   91.0    54.95   
claude-prediction-online                            248.0  432.0    57.41   
prediction-offline                                   19.0   32.0    59.38   
prediction-offline-sme                              106.0  183.0    57.92   
prediction-online                                   136.0  194.0    70.10   
prediction-online-sme                               313.0  647.0    48.38   
prediction-online-summarized-info                     1.0    2.0    50.00   
prediction-sentence-embedding-bold                    0.0    0.0      NaN   
prediction-sentence-embedding-conservative            0.0    0.0      NaN   

                                           confidence_0.7                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                           170.0   390.0    43.59   
claude-prediction-online                            454.0   811.0    55.98   
prediction-offline                                  162.0   291.0    55.67   
prediction-offline-sme                              343.0   505.0    67.92   
prediction-online                                  2075.0  3130.0    66.29   
prediction-online-sme                              2574.0  5249.0    49.04   
prediction-online-summarized-info                     7.0    13.0    53.85   
prediction-sentence-embedding-bold                   22.0    35.0    62.86   
prediction-sentence-embedding-conservative            0.0     0.0      NaN   

                                           confidence_0.8                    \
                                                n_correct   n_pred accuracy   
tool                                                                          
claude-prediction-offline                            57.0    104.0    54.81   
claude-prediction-online                             63.0    166.0    37.95   
prediction-offline                                  689.0   1596.0    43.17   
prediction-offline-sme                              893.0   1873.0    47.68   
prediction-online                                 10353.0  22531.0    45.95   
prediction-online-sme                              9090.0  17569.0    51.74   
prediction-online-summarized-info                    47.0    103.0    45.63   
prediction-sentence-embedding-bold                   16.0     17.0    94.12   
prediction-sentence-embedding-conservative           18.0     35.0    51.43   

                                           confidence_0.9                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                            77.0    

### Check the percentage of wins vs probability for all the tools

In [14]:
tools_stats_per_prob = gen_stats(tools_group, "win_probability")
tools_stats_per_prob

win_probability_0.5         \
                                                     n_correct n_pred   
tool                                                                    
claude-prediction-offline                                  0.0    0.0   
claude-prediction-online                                   0.0    0.0   
prediction-offline                                         0.0    0.0   
prediction-offline-sme                                     0.0    0.0   
prediction-online                                          0.0    0.0   
prediction-online-sme                                      0.0    0.0   
prediction-online-summarized-info                          0.0    0.0   
prediction-sentence-embedding-bold                         0.0    0.0   
prediction-sentence-embedding-conservative                 0.0    0.0   

                                                    win_probability_0.505  \
                                           accuracy             n_correct   
tool                                                                        
claude-prediction-offline                       NaN                   0.0   
claude-prediction-online                        NaN                   0.0   
prediction-offline                              NaN                   0.0   
prediction-offline-sme                          NaN                   0.0   
prediction-online                               NaN                   0.0   
prediction-online-sme                           NaN                   0.0   
prediction-online-summarized-info               NaN                   0.0   
prediction-sentence-embedding-bold              NaN                   0.0   
prediction-sentence-embedding-conservative      NaN                   0.0   

                                                            \
                                           n_pred accuracy   
tool                                                         
claude-prediction-offline                     0.0      NaN   
claude-prediction-online                      1.0      0.0   
prediction-offline                            0.0      NaN   
prediction-offline-sme                        0.0      NaN   
prediction-online                             0.0      NaN   
prediction-online-sme                         0.0      NaN   
prediction-online-summarized-info             0.0      NaN   
prediction-sentence-embedding-bold            0.0      NaN   
prediction-sentence-embedding-conservative    0.0      NaN   

                                           win_probability_0.525         \
                                                       n_correct n_pred   
tool                                                                      
claude-prediction-offline                                    0.0    0.0   
claude-prediction-online                                     0.0    0.0   
prediction-offline                                           0.0    0.0   
prediction-offline-sme                                       0.0    0.0   
prediction-online                                            0.0    0.0   
prediction-online-sme                                        0.0    1.0   
prediction-online-summarized-info                            0.0    0.0   
prediction-sentence-embedding-bold                           0.0    0.0   
prediction-sentence-embedding-conservative                   0.0    0.0   

                                                    win_probability_0.53  \
                                           accuracy            n_correct   
tool                                                                       
claude-prediction-offline                       NaN                  0.0   
claude-prediction-online                        NaN                  2.0   
prediction-offline                              NaN                  0.0   
prediction-offline-sme                          NaN                  0.0   
prediction-online                               NaN            

### List the well performing tools

In [15]:
# we are using 55% as the performance threshold and 100 as the samples threshold
perf_threshold = 55
samples_threshold = 100

In [16]:
well_performing = tools_stats_per_conf.loc[:, ("total", "accuracy")] > perf_threshold
enough_samples = tools_stats_per_conf.loc[:, ("total", "n_pred")] > samples_threshold
tools_stats_per_conf.loc[well_performing & enough_samples, [("total", "accuracy"), ("total", "n_pred")]]

total        
                          accuracy  n_pred
tool                                      
claude-prediction-offline    56.04  1697.0